In [1]:
import pandas as pd

In [2]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_mass_shootings_in_the_United_States"

In [38]:
#load the data from wikipedia
wiki_dfs = pd.read_html(wiki_url)

In [39]:
# remove the last item (wikipedia related items)
dont_care = wiki_dfs.pop()

In [40]:
#remove wiki's summary cells 
del wiki_dfs[13]
del wiki_dfs[13] #cell 14 becomes 13 after the last command, so it looks dumb

In [41]:
#fix the one table where they didn't label description
wiki_dfs[11]["Description"] = wiki_dfs[11]["Unnamed: 5"]
wiki_dfs[11].drop(columns=['Unnamed: 5'], inplace=True)

In [42]:
#join (union) all of  the dfs ( tables from the wikipedia table ) to one df
#list comprehension used in the concat function rather than listing all dfs manually
all_shootings_df  = pd.concat([  df for df in wiki_dfs ])

In [43]:
#set index to date
all_shootings_df = all_shootings_df.set_index("Date")

In [44]:
#split out the dead notes
all_shootings_df[['Dead New','Dead Notes 1','Dead Notes 2']] = all_shootings_df.Dead.str.split("[",expand=True)
all_shootings_df['Dead Notes 1'] = all_shootings_df['Dead Notes 1'].str.replace("]", "")
all_shootings_df['Dead Notes 2'] = all_shootings_df['Dead Notes 2'].str.replace("]", "")

In [45]:
# replace dead new with dead and split dead min and max
all_shootings_df['Dead'] = all_shootings_df['Dead New']

In [46]:
#drop dead new
all_shootings_df.drop(columns=['Dead New'], inplace=True)

In [47]:
#split dead min and max
all_shootings_df[['Dead Min','Dead Max']] = all_shootings_df.Dead.str.split("-",expand=True)

In [48]:
#Drop Dead 
all_shootings_df.drop(columns=['Dead'], inplace=True)

In [49]:
#split out the Injured notes
all_shootings_df[['Injured New','Injured Notes 1','Injured Notes 2']] = all_shootings_df.Injured.str.split("[",expand=True)
all_shootings_df['Injured Notes 1'] = all_shootings_df['Injured Notes 1'].str.replace("]", "")
all_shootings_df['Injured Notes 2'] = all_shootings_df['Injured Notes 2'].str.replace("]", "")
# replace dead new with dead and split dead min and max
all_shootings_df['Injured'] = all_shootings_df['Injured New']
all_shootings_df.drop(columns=['Injured New'], inplace=True)
#split Injured min and max
all_shootings_df[['Injured Min','Injured Max']] = all_shootings_df.Injured.str.split("-",expand=True)
#injured has some unknowns and a + on some columns
all_shootings_df['Injured Min'] = all_shootings_df['Injured Min'].str.replace("+", "")
all_shootings_df['Injured Min'] = all_shootings_df['Injured Min'].str.replace("unknown", "0")

#Drop Injured 
all_shootings_df.drop(columns=['Injured'], inplace=True)

In [53]:
#average the dead as float
all_shootings_df['Dead Average'] = all_shootings_df[['Dead Min','Dead Max']].astype(float).mean(axis=1)

In [54]:
#average the injured as float
all_shootings_df['Injured Average'] = all_shootings_df[['Injured Min','Injured Max']].astype(float).mean(axis=1)

In [55]:
all_shootings_df

Location  \
Date                                                
April 15, 2021              Indianapolis, Indiana   
April 7, 2021           Rock Hill, South Carolina   
March 31, 2021                 Orange, California   
March 22, 2021                  Boulder, Colorado   
March 16, 2021                   Atlanta, Georgia   
...                                           ...   
September 9, 1924                Hanapepe, Hawaii   
January 1923                 Levy County, Florida   
June 21–22, 1922                 Herrin, Illinois   
May 31 – June 1, 1921  Greenwood, Tulsa, Oklahoma   
November 2, 1920                   Ocoee, Florida   

                                              Total  \
Date                                                  
April 15, 2021                                   16   
April 7, 2021                                     7   
March 31, 2021                                    6   
March 22, 2021                                   11   
March 16, 2021                                    9   
...                                             ...   
September 9, 1924                                20   
January 1923                                 35-158   
June 21–22, 1922                                 23   
May 31 – June 1, 1921  836+ (exact number disputed)   
November 2, 1920                             34-37+   

                                                             Description  \
Date                                                                       
April 15, 2021         Indianapolis FedEx shooting: early reports sta...   
April 7, 2021          2021 Rock Hill shooting: 6 people, including 2...   
March 31, 2021         2021 Orange, California office shooting: 4 peo...   
March 22, 2021         2021 Boulder shooting: A mass shooting occurre...   
March 16, 2021         2021 Atlanta spa shootings: A series of mass s...   
...                                                                  ...   
September 9, 1924      Hanapepe Massacre: During a strike of Filipino...   
January 1923           Rosewood Massacre: The massacre was a racially...   
June 21–22, 1922       Herrin Massacre: During a United Mineworkers o...   
May 31 – June 1, 1921  Tulsa race massacre: When white mobs attacked ...   
November 2, 1920       Ocoee massacre: White mob attack on African-Am...   

                      Dead Notes 1 Dead Notes 2 Dead Min Dead Max  \
Date                                                                
April 15, 2021                 n 1         None        9     None   
April 7, 2021                  n 1         None        7     None   
March 31, 2021                None         None        4     None   
March 22, 2021                None         None       10     None   
March 16, 2021                None         None        8     None   
...                            ...          ...      ...      ...   
September 9, 1924             None         None       20     None   
January 1923                  None         None        8     None   
June 21–22, 1922              None         None       23     None   
May 31 – June 1, 1921          191          192      36      300    
November 2, 1920              None         None       34       37   

                      Injured Notes 1 Injured Notes 2 Injured Min Injured Max  \
Date                                                                            
April 15, 2021                   None            None           7        None   
April 7, 2021                    None            None           0        None   
March 31, 2021                    n 1            None           2        None   
March 22, 2021                    n 1            None           1        None   
March 16, 2021                   None            None           1        None   
...                               ...             ...         ...         ...   
September 9, 1924                None            None           0        None   
January 1923       

In [ ]:
#notes:
# - n1 in dead includes shooter, put in another column
# -nX in dead includes non-shooting related ( stabbing )- on the fence
# Dates - dates spanning multiple days will be limited to the start date
# location will get split to city / state as new columns
# where there is a range specified, the lowest of that range will be used 